In [1]:
from py_wake.wind_farm_models import All2AllIterative
from py_wake.deficit_models.deficit_model import WakeDeficitModel, BlockageDeficitModel
from py_wake.deficit_models.no_wake import NoWakeDeficit
from py_wake.site._site import UniformSite
from py_wake.flow_map import XYGrid
from py_wake.flow_map import XZGrid

from py_wake.turbulence_models import CrespoHernandez
from py_wake.turbulence_models import GCLTurbulence
from py_wake.deficit_models import Rathmann
from py_wake.deflection_models import JimenezWakeDeflection
from py_wake.deflection_models import GCLHillDeflection
from py_wake.utils.plotting import setup_plot

from py_wake.site import UniformWeibullSite
from py_wake.superposition_models import LinearSum

from py_wake.wind_turbines.generic_wind_turbines import GenericWindTurbine

from py_wake.deficit_models.gaussian import BlondelSuperGaussianDeficit2020
from py_wake.literature.gaussian_models import Blondel_Cathelain_2020

from scipy.optimize import differential_evolution
import matplotlib.pyplot as plt
import numpy as np
import os
import py_wake
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import math

from py_wake.wind_turbines import WindTurbine, WindTurbines

In [2]:
#turbine parameters
D = 283.21 #rotor diameter
H=170 #hub height
Prated=22000 #kW


WS=9 #wind speed
TI=0 #turbulence intensity

p_wd=[0.03597152, 0.03948682, 0.05167394999999999, 0.07000154, 0.08364547,0.0643485,0.08643194,0.1177051,0.1515757, 0.14737920000000002, 0.1001205 , 0.1001205]  # sector frequencies
#p_wd = np.ones(12) / 12
a=[9.176929, 9.782334, 9.531809,9.909545, 10.04269, 9.593921, 9.584007, 10.51499, 11.39895, 11.68746,11.63732,10.08803]         # Weibull scale parameter
k=[ 2.392578,2.447266,2.412109,2.591797, 2.755859, 2.595703, 2.583984,2.548828, 2.470703,2.607422, 2.626953, 2.326172]          # Weibull shape parameter

#site
from py_wake.site.xrsite import XRSite
import xarray as xr
site=XRSite(ds=xr.Dataset(data_vars={'Sector_frequency': ('wd', p_wd), 'Weibull_A': ('wd', a), 'Weibull_k': ('wd', k), 'TI': TI},
                  coords={'wd': np.linspace(0, 360, len(p_wd), endpoint=False)}))

from wind_turbine_object import create_wind_turbine 


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sko.GA import GA


wd=300
WD=270

alpha_degrees=wd-WD #deg
alpha_radians = np.radians(alpha_degrees)

coefficients_bs=np.loadtxt('coefficients_bs.txt')
coefficients_cs=np.loadtxt('coefficients_cs.txt')
coefficients_bf=np.loadtxt('coefficients_bf.txt')
coefficients_cf=np.loadtxt('coefficients_cf.txt')

# Define the polynomial function
def polynomial_function(coefficients, x):
    degree = len(coefficients) - 1
    f = sum(coef * x**(degree - i) for i, coef in enumerate(coefficients))
    return f


x1= 0
params = [
    polynomial_function(coefficients_bs, x1),
    polynomial_function(coefficients_cs, x1),
    polynomial_function(coefficients_bf, x1),
    polynomial_function(coefficients_cf, x1)
]

wind_turbine=create_wind_turbine(x1, D, H)


def evaluate (yaw): 
    yaw1,yaw2,yaw3,yaw4,yaw5=yaw

    # First turbine
    x_array = np.array([0])
    y_array = np.array([0])
    d_array = np.sqrt(x_array**2 + y_array**2)
    theta_array = np.arctan2(y_array, x_array) 
    new_x = d_array * np.cos(theta_array - alpha_radians)
    new_y = d_array * np.sin(theta_array - alpha_radians)

    wind_farm_model = All2AllIterative(site, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                        b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
    sim_res = wind_farm_model(new_x, new_y, yaw=yaw1, tilt=0,wd=WD, ws=WS)
    power1 = sim_res.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

    # Flow box for first turbine
    flow_box = wind_farm_model(new_x,new_y, yaw=yaw1,tilt=0,wd=np.arange(WD-1,WD+1)).flow_box(
        x=np.linspace(-D, 26*D, 101),
        y=np.linspace( - 6*D,  6*D, 101),
        h=H)
    wake_site = XRSite.from_flow_box(flow_box)

    # Second turbine
    x_array_2 = np.array([2.85 * D])
    y_array_2 = np.array([1.65*D])
    d_array_2 = np.sqrt(x_array_2**2 + y_array_2**2)
    theta_array_2 = np.arctan2(y_array_2, x_array_2) 
    new_x_2 = d_array_2 * np.cos(theta_array_2 - alpha_radians)
    new_y_2 = d_array_2 * np.sin(theta_array_2 - alpha_radians)
    wind_farm_model_2 = All2AllIterative(wake_site, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                        b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
    sim_res_2 = wind_farm_model_2(new_x_2, new_y_2, yaw=yaw2, tilt=0,wd=WD, ws=WS)
    power2 = sim_res_2.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

    # Flow box for second turbine
    flow_box_2 = wind_farm_model_2(new_x_2,new_y_2, yaw=yaw2,tilt=0,wd=np.arange(WD-1,WD+1)).flow_box(
        x=np.linspace(-D, 26*D , 101),
        y=np.linspace(0 - 6*D,  6*D , 101),
        h=H)
    wake_site_2 = XRSite.from_flow_box(flow_box_2)

    # Third turbine
    x_array_3 = np.array([5.7*D])
    y_array_3 = np.array([3.3*D])
    d_array_3 = np.sqrt(x_array_3**2 + y_array_3**2)
    theta_array_3 = np.arctan2(y_array_3, x_array_3) 
    new_x_3 = d_array_3 * np.cos(theta_array_3 - alpha_radians)
    new_y_3 = d_array_3 * np.sin(theta_array_3 - alpha_radians)
    wind_farm_model_3 = All2AllIterative(wake_site_2, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                        b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
    sim_res_3 = wind_farm_model_3(new_x_3, new_y_3, yaw=yaw3, tilt=0,wd=WD, ws=WS)
    power3 = sim_res_3.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

    # Flow box for third turbine
    flow_box_3 = wind_farm_model_3(new_x_3, new_y_3,yaw=yaw3,tilt=0, wd=np.arange(WD-1,WD+1)).flow_box(
        x=np.linspace(-D, 26*D - 1, 101),
        y=np.linspace(0 - 6*D,  6*D , 101),
        h=H)
    wake_site_3 = XRSite.from_flow_box(flow_box_3)

    # Fourth turbine
    x_array_4 = np.array([8.58*D])
    y_array_4 = np.array([-0.05*D])
    d_array_4 = np.sqrt(x_array_4**2 + y_array_4**2)
    theta_array_4 = np.arctan2(y_array_4, x_array_4) 
    new_x_4 = d_array_4 * np.cos(theta_array_4 - alpha_radians)
    new_y_4 = d_array_4 * np.sin(theta_array_4 - alpha_radians)
    wind_farm_model_4 = All2AllIterative(wake_site_3, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                        b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
    sim_res_4 = wind_farm_model_4(new_x_4, new_y_4, yaw=yaw4, tilt=0,wd=WD, ws=WS)
    power4 = sim_res_4.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6
    

    # Flow box for forth turbine
    flow_box_4 = wind_farm_model_4(new_x_4,new_y_4,yaw=yaw4,tilt=0, wd=np.arange(WD-1,WD+1)).flow_box(
    x=np.linspace(-D, 26*D -1, 101),
    y=np.linspace(0 - 6*D, 0 + 6*D, 101),
    h=H)
    wake_site_4 = XRSite.from_flow_box(flow_box_4)
    

    # Fifth turbine
    x_array_5 = np.array([ 4*2.85 * D])
    y_array_5 = np.array([ 1.6*D])
    d_array_5 = np.sqrt(x_array_5**2 + y_array_5**2)
    theta_array_5 = np.arctan2(y_array_5, x_array_5) 
    new_x_5 = d_array_5 * np.cos(theta_array_5 - alpha_radians)
    new_y_5 = d_array_5 * np.sin(theta_array_5 - alpha_radians)
    wind_farm_model_5 = All2AllIterative(wake_site_4, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                    b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(), deflectionModel=GCLHillDeflection(),turbulenceModel=GCLTurbulence())
    sim_res_5 = wind_farm_model_5(new_x_5, new_y_5, yaw=yaw5, tilt=0,wd=WD, ws=WS)
    power5=sim_res_5.Power.sel(ws=WS,wt=0, wd=WD).sum().values/1e6
    wind5 = sim_res_5.flow_map(grid=XYGrid(x=np.linspace(-D, (26*D-1), int(26*0.1*D)), y=np.linspace(-int(6*D), int(6*D), int(2*6*D/10)),h=H)).WS_eff.values.squeeze()

    total_power = power1 + power2 + power3 + power4+ power5



    # Return negative total power because GA in sko minimizes the function
    return total_power
    
yaw_range = np.arange(-15, 15, 6) 
yaw3=0
yaw5=0
max_power=0
for yaw1 in yaw_range:
    for yaw2 in yaw_range:
        for yaw4 in yaw_range:
            yaw = [yaw1, yaw2, yaw3, yaw4, yaw5]
            power = evaluate(yaw)
            if power > max_power:
                max_power = power
                best_yaw = yaw

# First turbine
yaw1=best_yaw[0]
yaw2=best_yaw[1]
yaw4=best_yaw[3]


x_array = np.array([0])
y_array = np.array([0])
d_array = np.sqrt(x_array**2 + y_array**2)
theta_array = np.arctan2(y_array, x_array) 
new_x = d_array * np.cos(theta_array - alpha_radians)
new_y = d_array * np.sin(theta_array - alpha_radians)

wind_farm_model = All2AllIterative(site, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                    b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
sim_res = wind_farm_model(new_x, new_y, yaw=yaw1, tilt=0,wd=WD, ws=WS)
power1 = sim_res.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

# Flow box for first turbine
flow_box = wind_farm_model(new_x,new_y, yaw=yaw1,tilt=0,wd=np.arange(WD-1,WD+1)).flow_box(
    x=np.linspace(-D, 26*D, 101),
    y=np.linspace( - 6*D,  6*D, 101),
    h=H)
wake_site = XRSite.from_flow_box(flow_box)

# Second turbine
x_array_2 = np.array([2.85 * D])
y_array_2 = np.array([1.65*D])
d_array_2 = np.sqrt(x_array_2**2 + y_array_2**2)
theta_array_2 = np.arctan2(y_array_2, x_array_2) 
new_x_2 = d_array_2 * np.cos(theta_array_2 - alpha_radians)
new_y_2 = d_array_2 * np.sin(theta_array_2 - alpha_radians)
wind_farm_model_2 = All2AllIterative(wake_site, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                    b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
sim_res_2 = wind_farm_model_2(new_x_2, new_y_2, yaw=yaw2, tilt=0,wd=WD, ws=WS)
power2 = sim_res_2.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

# Flow box for second turbine
flow_box_2 = wind_farm_model_2(new_x_2,new_y_2,yaw=yaw2,tilt=0, wd=np.arange(WD-1,WD+1)).flow_box(
    x=np.linspace(-D, 26*D , 101),
    y=np.linspace(0 - 6*D,  6*D , 101),
    h=H)
wake_site_2 = XRSite.from_flow_box(flow_box_2)

# Third turbine
x_array_3 = np.array([5.7*D])
y_array_3 = np.array([3.3*D])
d_array_3 = np.sqrt(x_array_3**2 + y_array_3**2)
theta_array_3 = np.arctan2(y_array_3, x_array_3) 
new_x_3 = d_array_3 * np.cos(theta_array_3 - alpha_radians)
new_y_3 = d_array_3 * np.sin(theta_array_3 - alpha_radians)
wind_farm_model_3 = All2AllIterative(wake_site_2, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                    b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
sim_res_3 = wind_farm_model_3(new_x_3, new_y_3, yaw=yaw3, tilt=0,wd=WD, ws=WS)
power3 = sim_res_3.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6

# Flow box for third turbine
flow_box_3 = wind_farm_model_3(new_x_3, new_y_3,yaw=yaw3,tilt=0, wd=np.arange(WD-1,WD+1)).flow_box(
    x=np.linspace(-D, 26*D - 1, 101),
    y=np.linspace(0 - 6*D,  6*D , 101),
    h=H)
wake_site_3 = XRSite.from_flow_box(flow_box_3)

# Fourth turbine
x_array_4 = np.array([8.58*D])
y_array_4 = np.array([-0.05*D])
d_array_4 = np.sqrt(x_array_4**2 + y_array_4**2)
theta_array_4 = np.arctan2(y_array_4, x_array_4) 
new_x_4 = d_array_4 * np.cos(theta_array_4 - alpha_radians)
new_y_4 = d_array_4 * np.sin(theta_array_4 - alpha_radians)
wind_farm_model_4 = All2AllIterative(wake_site_3, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                    b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
sim_res_4 = wind_farm_model_4(new_x_4, new_y_4, yaw=yaw4, tilt=0,wd=WD, ws=WS)
power4 = sim_res_4.Power.sel(ws=WS, wt=0, wd=WD).sum().values / 1e6


# Flow box for forth turbine
flow_box_4 = wind_farm_model_4(new_x_4,new_y_4,yaw=yaw4,tilt=0, wd=np.arange(WD-1,WD+1)).flow_box(
x=np.linspace(-D, 26*D -1, 101),
y=np.linspace(0 - 6*D, 0 + 6*D, 101),
h=H)
wake_site_4 = XRSite.from_flow_box(flow_box_4)


# Fifth turbine
x_array_5 = np.array([ 4*2.85 * D])
y_array_5 = np.array([ 1.6*D])
d_array_5 = np.sqrt(x_array_5**2 + y_array_5**2)
theta_array_5 = np.arctan2(y_array_5, x_array_5) 
new_x_5 = d_array_5 * np.cos(theta_array_5 - alpha_radians)
new_y_5 = d_array_5 * np.sin(theta_array_5 - alpha_radians)
wind_farm_model_5 = All2AllIterative(wake_site_4, wind_turbine, wake_deficitModel=BlondelSuperGaussianDeficit2020(a_s=0, b_s=params[0], c_s=params[1],
                b_f=params[2], c_f=params[3], use_effective_ws=True), blockage_deficitModel=Rathmann(),deflectionModel=GCLHillDeflection(), turbulenceModel=GCLTurbulence())
sim_res_5 = wind_farm_model_5(new_x_5, new_y_5, yaw=yaw5, tilt=0,wd=WD, ws=WS)
power5=sim_res_5.Power.sel(ws=WS,wt=0, wd=WD).sum().values/1e6
wind5 = sim_res_5.flow_map(grid=XYGrid(x=np.linspace(-D, (26*D-1), int(26*0.1*D)), y=np.linspace(-int(6*D), int(6*D), int(2*6*D/10)),h=H)).WS_eff.values.squeeze()

# Define the positions and values to be plotted
x_positions = [new_x[0]+50, new_x_2[0]/10+50, new_x_3[0]/10+50, new_x_4[0]/10+50, new_x_5[0]/10+50]  # Replace with actual positions
y_positions = [new_y[0]+130, new_y_2[0]/10+130, new_y_3[0]/10+130, new_y_4[0]/10+130, new_y_5[0]/10+130]  # Replace with actual positions
values = [round(power1, 2), round(power2, 2), round(power3, 2), round(power4, 2), round(power5, 2)]  # Replace with actual values
names = ["D08", "C08", "B08", "B07", "A07"]  # List of turbine names in order


fig = plt.figure(figsize=(16, 6))
im = plt.imshow(wind5, cmap='Blues', interpolation='nearest', vmin=0, vmax=WS)
plt.colorbar(im, label='wind speeds [m/s]')  # Add colorbar with label
plt.title(f'Lillgrund wind farm - Wind direction = \n{wd}°', fontsize=12, fontweight='bold')
plt.xlabel('x [m]', fontsize=12)
plt.ylabel('y [m]', fontsize=12)

filename = f'power_{wd}_wake_steering.txt'
np.savetxt(filename, values)

